# Datawrapper - making the charts with USGS data

- chart of population exposed by shake intensity
- chart of aftershocks

In [ ]:
import pandas as pd
from datawrapper import Datawrapper
import os
import geopandas as gpd

# Set Datawrapper with API key

In [ ]:
# Récupère ta clé API depuis les variables d'environnement
api_key = os.getenv("DATAWRAPPER_API_KEY")  # remplace par le nom exact de ta variable

# Initialise Datawrapper
dw = Datawrapper(api_key)



# # Tu peux maintenant utiliser dw.account_info()
# info = dw.account_info()
# print(info)

This method is deprecated and will be removed in a future version. Use get_account_info instead.


{'id': 489654, 'email': 'olaf.koenig@tamedia.ch', 'name': 'Olaf König', 'role': 'editor', 'language': 'en-US', 'presenceColor': 'color-dw-presence-30', 'avatar': None, 'customAvatar': None, 'teams': [{'id': 'tagesanzeiger', 'name': 'Tamedia', 'url': '/v3/teams/tagesanzeiger', 'active': False}], 'chartCount': 673, 'url': '/v3/users/489654', 'activeTeam': None, 'entitlements': {}}


In [ ]:
input_dir = "data_input/USGS"

### Data population exposure

In [ ]:
# Import data
df_exposure = pd.read_csv(f"{input_dir}/exposure_population_by_country_mmi.csv")
label_exposure = pd.read_csv(f"{input_dir}/mmi_intensity_category.csv")
print(df_exposure.head())
print(label_exposure.head())

df_exposure = pd.merge(
    df_exposure,
    label_exposure,
    how="left",
    left_on="mmi",
    right_on="Intensity"
)

df_exposure

  country  mmi  pop_exposure country_name_en country_name_fr country_name_de
0     ALL    1             0           Total           Total           Total
1     ALL    2             0           Total           Total           Total
2     ALL    3      32989235           Total           Total           Total
3     ALL    4     163708636           Total           Total           Total
4     ALL    5      24656409           Total           Total           Total
   Intensity Shaking_EN   Damage_EN     Shaking_DE    Damage_DE    Shaking_FR  \
0          1   Not felt         NaN  Nicht fühlbar        Keine  Non ressenti   
1          2       Weak         NaN        Schwach        Keine        Faible   
2          3       Weak         NaN        Schwach        Keine        Faible   
3          4      Light         NaN         Leicht        Keine         Léger   
4          5   Moderate  Very light         Mässig  Sehr gering        Modéré   

    Damage_FR  
0       Aucun  
1       Aucun  
2  

,country,mmi,pop_exposure,country_name_en,country_name_fr,country_name_de,Intensity,Shaking_EN,Damage_EN,Shaking_DE,Damage_DE,Shaking_FR,Damage_FR
0,ALL,1,0,Total,Total,Total,1,Not felt,NaN,Nicht fühlbar,Keine,Non ressenti,Aucun
1,ALL,2,0,Total,Total,Total,2,Weak,NaN,Schwach,Keine,Faible,Aucun
2,ALL,3,32989235,Total,Total,Total,3,Weak,NaN,Schwach,Keine,Faible,Aucun
3,ALL,4,163708636,Total,Total,Total,4,Light,NaN,Leicht,Keine,Léger,Aucun
4,ALL,5,24656409,Total,Total,Total,5,Moderate,Very light,Mässig,Sehr gering,Modéré,Très léger
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,TH,6,141478,Thailand,Thaïlande,Thailand,6,Strong,Light,Stark,Gering,Fort,Léger
96,TH,7,0,Thailand,Thaïlande,Thailand,7,Very strong,Moderate,Sehr stark,Mittel,Très fort,Modéré
97,TH,8,0,Thailand,Thaïlande,Thailand,8,Severe,Moderate/heavy,Heftig,Mittel/hoch,Sévère,Modéré/important
98,TH,9,0,Thailand,Thaïlande,Thailand,9,Violent,Heavy,Heftig,Hoch,Violent,Important


### Cleaning exposure and transform to wide

In [111]:
import pandas as pd

# 1. Pivot large
df_wide = (
    df_exposure[["mmi", "Shaking_EN", "country_name_en", "pop_exposure"]]
    .pivot(index=["mmi", "Shaking_EN"], columns="country_name_en", values="pop_exposure")
    .reset_index()
)

# 2. Supprimer les intensités faibles
df_wide = df_wide[~df_wide["mmi"].isin([1, 2])]

# 3. Liste des pays (hors colonnes de base)
pays_cols = df_wide.columns.difference(["mmi", "Shaking_EN", "Total"])

# 4. Créer colonnes pourcentages
pct_col_names = {col: col.capitalize() for col in pays_cols}
for col, new_col in pct_col_names.items():
    df_wide[new_col] = (
        df_wide[col] / df_wide["Total"].replace(0, pd.NA) * 100
    ).clip(upper=100).round(1)

# 5. Pondérer l’exposition aux fortes intensités (e.g. mmi * pourcentage)
df_scores = df_wide[["mmi"] + list(pct_col_names.values())].copy()
for col in pct_col_names.values():
    df_scores[col] = df_scores["mmi"] * df_scores[col]

# 6. Score final par pays (somme pondérée), sauf Birmanie
exposure_scores = (
    df_scores[df_scores["mmi"] >= 4]  # facultatif : focus haute intensité
    .drop(columns=["mmi"])
    .sum()
    .sort_values(ascending=False)
)

# 7. Ordre des colonnes de % : Myanmar d’abord, puis autres selon score
pct_cols_ordered = ["Myanmar (burma)"] + [
    col for col in exposure_scores.index if col != "Myanmar (burma)"
]

# 8. Colonnes absolues renommées avec suffixe _abs
for col in pays_cols:
    df_wide[col + "_abs"] = df_wide[col]

# 9. Colonnes absolues dans le même ordre
abs_cols_ordered = [
    key + "_abs"
    for key, val in pct_col_names.items()
    if val in pct_cols_ordered
]

# 10. Colonnes finales dans le bon ordre
final_cols = ["mmi", "Shaking_EN", "Total"] + pct_cols_ordered + abs_cols_ordered
df_wide = df_wide[final_cols]

# 11. Trier les lignes : MMI décroissant, % Myanmar décroissant
df_wide = df_wide.sort_values(by=["mmi", "Myanmar (burma)"], ascending=[False, False])




### Styling for datawrapper

In [112]:
# Mapping couleur : du jaune clair au rouge foncé
intensity_colors = {
    "Weak":        "#fff5f0",
    "Light":       "#fee0d2",
    "Moderate":    "#fcbba1",
    "Strong":      "#fc9272",
    "Very strong": "#fb6a4a",
    "Severe":      "#ef3b2c",
    "Violent":     "#cb181d",
    "Extreme":     "#99000d"
}

# Crée directement Intensity_html basé sur Shaking_EN + couleur
df_wide["Intensity_html"] = df_wide["Shaking_EN"].map(
    lambda val: f'<span style="background-color: {intensity_colors.get(val, "#cccccc")}; color: white; padding: 2px 6px; border-radius: 4px;">{val}</span>'
)


In [96]:
df_wide

country_name_en,mmi,Shaking_EN,Total,Myanmar (burma),Thailand,Bangladesh,India,China,Laos,Bhutan,...,Bangladesh_abs,Bhutan_abs,Cambodia_abs,China_abs,India_abs,Laos_abs,Myanmar (Burma)_abs,Thailand_abs,Vietnam_abs,Intensity_html
9,10,Extreme,414753,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,414753,0.0,0.0,"<span style=""background-color: #800026; color:..."
8,9,Violent,5800931,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6215684,0.0,0.0,"<span style=""background-color: #bd0026; color:..."
7,8,Severe,3636635,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3636635,0.0,0.0,"<span style=""background-color: #e31a1c; color:..."
6,7,Very strong,8787774,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8787774,0.0,0.0,"<span style=""background-color: #fc4e2a; color:..."
5,6,Strong,20277188,98.7,0.7,0.0,0.0,0.6,0.0,0.0,...,0.0,0.0,0.0,0.6,0.0,0.0,20015152,0.7,0.0,"<span style=""background-color: #fd8d3c; color:..."
4,5,Moderate,24656409,53.7,38.9,0.0,5.9,1.5,0.0,0.0,...,0.0,0.0,0.0,1.5,5.9,0.0,13251671,38.9,0.0,"<span style=""background-color: #feb24c; color:..."
3,4,Light,163708636,3.1,20.9,42.0,22.5,10.1,1.4,0.0,...,42.0,0.0,0.0,10.1,22.5,1.4,5153110,20.9,0.0,"<span style=""background-color: #ffeda0; color:..."
2,3,Weak,32989235,0.0,16.2,20.4,1.6,49.0,5.2,0.0,...,20.4,0.0,5.3,49.0,1.6,5.2,6815,16.2,2.2,"<span style=""background-color: #ffffcc; color:..."


### Create dw exposure chart

In [ ]:
# Folder ID
folder_id = "332403"

Function create chart (run one time)

In [ ]:
# # The function create_chart() can take four arguments: title, chart_type, data, and folder_id.

# dw_exposure = dw.create_chart(
#     title="Population exposure at the earthquake",
#     chart_type="tables",
#     data = df_wide,
#     folder_id=folder_id
# )

### Assign ID to chart

In [22]:
# dw_exposure_id = dw_exposure['id']
# print(dw_exposure_id)

dw_exposure_id = "dQpEY"

### Update data

In [114]:
# Mettre Intensity_html en première colonne (à la place de mmi)
cols = ["Intensity_html"] + [col for col in df_wide.columns if col not in ["mmi", "Intensity_html"]]
df_wide = df_wide[cols]
dw.add_data(dw_exposure_id, df_wide)


True

### Publish chart

In [83]:
dw.publish_chart(dw_exposure_id)

dw_exposure_iframe = dw.get_iframe_code(dw_exposure_id)

Post request failed with status code 400.


FailedRequest: Request failed with status code 400. Response content: b'{"statusCode":400,"error":"Bad Request","message":"Invalid request payload input: \\"value\\" must be of type object","type":"error / validation","details":[{"path":"","type":"object.base"}]}'

## Aftershocks data

In [ ]:
dw_aftershocks = gpd.read_file(f"{input_dir}/aftershock_with_distance.gpkg")
dw_aftershocks

,id,time,magnitude,depth,latitude,longitude,magtype,gap,rms,horizontal_error,vertical_error,distance_to_fault_m,distance_to_fault_km,geometry
0,us7000pn9s,2025-03-28 06:20:52.715000+00:00,7.7,10.0,22.0110,95.9363,None,None,None,None,None,4.841912e+03,4.841912,POINT (95.9363 22.011)
1,us7000pn9z,2025-03-28 06:32:04.777000+00:00,6.7,10.0,21.6975,95.9690,None,None,None,None,None,2.036882e+03,2.036882,POINT (95.969 21.6975)
2,us7000pncy,2025-03-28 06:39:14.645000+00:00,4.8,10.0,19.8979,95.8026,None,None,None,None,None,3.629403e+04,36.294033,POINT (95.8026 19.8979)
3,us7000pncv,2025-03-28 06:42:24.760000+00:00,4.9,10.0,21.8377,95.8747,None,None,None,None,None,1.157123e+04,11.571228,POINT (95.8747 21.8377)
4,us7000pnb6,2025-03-28 06:45:44.906000+00:00,4.9,10.0,19.1284,96.2075,None,None,None,None,None,6.474854e+03,6.474854,POINT (96.2075 19.1284)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,us6000q72i,2025-04-18 18:01:22.215000+00:00,4.1,10.0,20.4354,96.1272,None,None,None,None,None,4.795000e+03,4.795000,POINT (96.1272 20.4354)
70,us6000q7gc,2025-04-21 00:03:23.936000+00:00,4.2,10.0,29.1753,86.9720,None,None,None,None,None,1.168997e+06,1168.996823,POINT (86.972 29.1753)
71,us6000q7ge,2025-04-21 01:07:50.979000+00:00,4.7,10.0,32.5723,93.3067,None,None,None,None,None,1.149769e+06,1149.768632,POINT (93.3067 32.5723)
72,us7000puj0,2025-04-23 16:09:01.885000+00:00,4.9,10.0,26.4164,100.0261,None,None,None,None,None,6.011740e+05,601.174035,POINT (100.0261 26.4164)


### Filter earthquakes with minimum distance to fault

In [ ]:
# Filtering distance
distance_to_fault = 100

dw_aftershocks_filtered = dw_aftershocks[dw_aftershocks["distance_to_fault_km"] <= distance_to_fault]

### Clean time and accentuate sizes differences

In [120]:

df_aftershocks_filtered = dw_aftershocks_filtered.drop(columns="geometry").copy()

# Conversion en datetime, si nécessaire
df_aftershocks_filtered["time"] = pd.to_datetime(df_aftershocks_filtered["time"])

# Chronologie fine pour DW
df_aftershocks_filtered["datetime"] = df_aftershocks_filtered["time"].dt.strftime('%Y-%m-%d %H:%M')
df_aftershocks_filtered["iso8601"] = df_aftershocks_filtered["time"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

# Ajout du radius avec une fonction quadratique
def mag_to_radius(mag, base=2):
    return base + (mag - 3.5)**2.5

df_aftershocks_filtered["radius"] = df_aftershocks_filtered["magnitude"].apply(mag_to_radius)

# Sélection des colonnes clés
cols = ["id", "magnitude", "radius", "distance_to_fault_km", "datetime", "iso8601"]
df_aftershock_for_dw = df_aftershocks_filtered[cols].copy()



### Create chart for aftershocks

In [ ]:
# dw_aftershock = dw.create_chart(
#     title="Aftershocks",
#     chart_type="tables",
#     data = df_aftershock_for_dw,
#     folder_id=folder_id
# )

In [121]:
# dw_aftershock_id = dw_aftershock['id']
# print(dw_aftershock_id)

dw_aftershock_id = "UWXHh"

### Upadate chart data

In [ ]:
dw.add_data(dw_aftershock_id, df_aftershock_for_dw)

True

### Count aftershocks in a time window

In [125]:
# =============================================================================
# =============================================================================
# --- 1. Paramètres utilisateur ---
days_after = 6
# =============================================================================
# =============================================================================


# S'assurer que le DataFrame est trié par 'time' croissante
df_aftershocks_filtered = df_aftershocks_filtered.sort_values("time")

# Prendre la date du tout premier aftershock comme point de départ
first_time = df_aftershocks_filtered["time"].iloc[0]

# Définir la fenêtre de 6 jours après ce timestamp
window_end = first_time + pd.Timedelta(days=days_after)

# Filtrer les aftershocks dans cette fenêtre
after_6days = df_aftershocks_filtered[
    (df_aftershocks_filtered["time"] >= first_time) &
    (df_aftershocks_filtered["time"] < window_end)
]

count = len(after_6days)
print(f"Nombre de tremblements de terre dans les {days_after} jours à partir du premier : {count}")


Nombre de tremblements de terre dans les 6 jours à partir du premier : 38
